In [2]:
## pip install dspy-ai[chromadb]

import chromadb
from chromadb.utils import embedding_functions
from dspy.retrieve.chromadb_rm import ChromadbRM

## example of chormadb
chroma_client = client = chromadb.PersistentClient(path="./furniture_example")
default_ef = embedding_functions.DefaultEmbeddingFunction()
collection = chroma_client.get_or_create_collection(name="furniture", embedding_function=default_ef)

collection.add(
    documents=[
        "couch, bed, table, chair", 
        "computer, server, table, chair"],
    metadatas=[
        {"source": "Bedroom"}, 
        {"source": "Office"}
        ],
    ids=[
        "id1", 
        "id2"
    ]
)

rm = ChromadbRM(collection_name='furniture', persist_directory="./furniture_example", embedding_function=default_ef)
print(rm('comfy'))

[{'long_text': 'couch, bed, table, chair'}, {'long_text': 'computer, server, table, chair'}]


In [3]:
import dspy

lm = dspy.OpenAI(model="gpt-3.5-turbo")
dspy.settings.configure(lm=lm, rm=rm)

In [4]:
from dspy import Signature, InputField, OutputField, Module, Retrieve, ChainOfThought, Prediction

# Define a signature for generating answers based on the question and context
class GenerateAnswer(Signature):
    context = InputField(desc="Context containing relevant facts.")
    question = InputField()
    answer = OutputField(desc="The generated answer, often between 1 and 5 words.")

# Define the RAG module
class RAG(Module):
    def __init__(self, num_passages=3):
        super().__init__()
        self.retrieve = Retrieve(k=num_passages)  # Number of passages to retrieve
        self.generate_answer = ChainOfThought(GenerateAnswer)
    
    def forward(self, question):
        # Use the Retrieve component to get relevant context
        context = self.retrieve(question).passages
        # Generate an answer based on the retrieved context and the question
        prediction = self.generate_answer(context=context, question=question)
        return Prediction(context=context, answer=prediction.answer)

# Instantiate the RAG system
rag_system = RAG()

# Example question
question = "What are things in office?"

# Activate the RAG system with your question
prediction = rag_system(question)

# Print the contexts and the generated answer
print(f"Question: {question}")
print(f"Predicted Answer: {prediction.answer}")
print(f"Retrieved Contexts (truncated): {[c[:200] + '...' for c in prediction.context]}")

Question: What are things in office?
Predicted Answer: computer, server, table, chair
Retrieved Contexts (truncated): ['computer, server, table, chair...', 'couch, bed, table, chair...']
